In [1]:
import pandas as pd
import os

In [2]:
# Define paths
response_folder = r'C:\Users\boatt\Desktop\Data-CTC\notebooks\Pivoteo de response\NocoDB\response'
id_key_file = r'C:\Users\boatt\Desktop\Data-CTC\notebooks\Pivoteo de response\NocoDB\id_key.csv'

In [3]:
# 1. Anexar todos los .csv de la carpeta response
all_files = [os.path.join(response_folder, f) for f in os.listdir(response_folder) if f.endswith('.csv')]

# Leer y concatenar los archivos CSV
df_list = [pd.read_csv(file) for file in all_files]
merged_df = pd.concat(df_list, axis=0)

In [4]:
#merged_df = merged_df[(merged_df['projectId'] == 19) & ((merged_df['tipo_linea'] != 'Salida') | (merged_df['tipo_linea'].isna()))] #Entrada
merged_df = merged_df[(merged_df['projectId'] == 19) & ((merged_df['tipo_linea'] == 'Salida'))] #Salida

In [5]:
merged_df['grade_section'] = merged_df['grado_actual'].astype(str) + '-' + merged_df['seccion'].astype(str)

In [6]:
cols = list(merged_df.columns)

# Quitar las columnas que vas a mover
cols.remove('nacionalidad')
cols.remove('ubicacion')
cols.remove('grade_section')

# Insertar 'nacionalidad' y 'ubicacion' después de 'nombre_completo'
idx_nombre = cols.index('nombre_completo') + 1
cols[idx_nombre:idx_nombre] = ['nacionalidad', 'ubicacion']

# Insertar 'grade_section' después de 'institucion_educativa'
idx_institucion = cols.index('institucion_educativa') + 1
cols[idx_institucion:idx_institucion] = ['grade_section']

# Reordenar el DataFrame
merged_df = merged_df[cols]

In [7]:
print(merged_df['tipo_linea'].value_counts())
print(merged_df['projectId'].value_counts())

tipo_linea
Salida    2467
Name: count, dtype: int64
projectId
19    2467
Name: count, dtype: int64


In [8]:
len(merged_df)

2467

In [9]:
merged_df.drop(columns=['Id', 'CreatedAt', 'UpdatedAt', 'grado_actual', 'seccion','tipo_grupo', 'tipo_linea', 'projectId'], inplace=True)

# eliminar las columnas vacias
merged_df.dropna(axis=1, how='all', inplace=True)

# eliminar duplicados
merged_df.drop_duplicates(inplace=True)

In [10]:
len(merged_df)

2464

In [11]:
# 2. Cambiar el nombre de las columnas
id_key_df = pd.read_csv(id_key_file)

# Crear un diccionario de mapeo de 'question' a 'question_key'
# Asegurarse de que no haya valores nulos en las columnas clave
id_key_df = id_key_df.dropna(subset=['question', 'question_key'])
rename_dict = pd.Series(id_key_df.question.values, index=id_key_df.question_key).to_dict()

# Renombrar las columnas del DataFrame fusionado
merged_df.rename(columns=rename_dict, inplace=True)

In [12]:
merged_df.to_excel('cuestionario de salida_19.xlsx', index=False)